In [2]:
import sys

sys.path.append("../../src/groundstate_prep")
from fuzzy_bisection import fuzzy_bisection
from ground_state_prep import prepare_ground_state

sys.path.append("../../src/lindbladian")
from lindbladian import circuit_implementation_lindbladian

In [3]:
# Hamiltonian.

import numpy as np
from numpy import linalg as LA
import qib
import matplotlib.pyplot as plt

# Parameters for the Ising Hamiltonian
# L has to be even! Due to K only being able to control even Ls!
L, J, g = (6, 1, 1)


# construct Hamiltonian
latt = qib.lattice.IntegerLattice((L,), pbc=True)
field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
hamil = qib.IsingHamiltonian(field, J, 0, g).as_matrix().toarray()

eigenvalues, eigenvectors = LA.eig(hamil)
idx = eigenvalues.argsort()
eigenvalues_sort = eigenvalues[idx]
eigenvectors_sort = eigenvectors[:,idx]
ground_state = eigenvectors_sort[:, 0]
print("Ground State Energy", eigenvalues_sort[0].real)

dist = 0.001
max_spectrum_length = 16
ground_energy_lower_bound = -9
c1 = (np.pi-2*dist) / (max_spectrum_length)
c2 = dist - c1 * ground_energy_lower_bound

eigenvalues_tr = eigenvalues_sort * c1 + c2
a_values = np.array([np.cos(eig/2) for eig in eigenvalues_tr])

print("a_max", a_values[0])
print("a_premax", a_values[1])
print("c1: ", c1)
print("c2: ", c2)

Ground State Energy -7.727406610312517
a_max (0.9921530917736707-0j)
a_premax (0.9885924745552997-0j)
c1:  0.19622454084936208
c2:  1.7670208676442587


In [4]:
mu, d, c, phis_max_iter, = (0.99, 30, 0.95, 10)

tau, nsteps = (1, 1500)
end_state_lind, en_list, err = circuit_implementation_lindbladian(hamil, L, tau=tau, nsteps=nsteps)

ket_0 = np.array([1,0])
end_state_qetu, E = prepare_ground_state(
                        np.kron(ket_0, end_state_lind), mu, d, c, phis_max_iter,
                        np.kron(ket_0, ground_state), L, J, g, eigenvalues_sort[0],
                        hamil=hamil, max_reps=5, tau=c1, shift=c2, a_max=a_values[0]
)


Layer 0
Prob 0: 0.08339700191905328
Prob 1: 0.916602998080947

Layer 1
Prob 0: 0.09804355119790571
Prob 1: 0.9019564488020938

Layer 2
Prob 0: 0.10116326035064872
Prob 1: 0.8988367396493514

Layer 3
Prob 0: 0.1022018381316293
Prob 1: 0.8977981618683705

Layer 4
Prob 0: 0.10253502829533823
Prob 1: 0.8974649717046618

F(a_max) = (0.1026838363758513+0j)

 ---------- 
 SUCCESS! 

Fidelity of the initial state to the ground state: 0.7585850631971753
Fidelity of the prepared state to the ground state: 0.999310886804061


In [5]:
a_est = fuzzy_bisection(end_state_qetu, 0, 1, 14, 0.005, 0, hamil, c1, c2, a_values[0])
print("Estimated a: " + str(a_est))
print("a: " + str(a_values[0]))

------------------
x: 0.5
d:  14
left:  0
right:  1
Prob 0: 0.9403862913465652
Prob 1: 0.05961370865343385
F(a_max)**2:  (0.9404807454460484+0j)
Success Prob:  0.9403862913465652
------------------
x: 0.745
d:  14
left:  0.49
right:  1


C:\Users\erena\AppData\Local\Programs\Python\Python310\lib\site-packages\cvxpy\problems\problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Prob 0: 0.858729295834927
Prob 1: 0.14127070416507465
F(a_max)**2:  (0.8588381830267405+0j)
Success Prob:  0.858729295834927
------------------
x: 0.8674999999999999
d:  14
left:  0.735
right:  1
Prob 0: 0.8585642512872751
Prob 1: 0.1414357487127249
F(a_max)**2:  (0.8586248916517395+0j)
Success Prob:  0.8585642512872751
------------------
x: 0.92875
d:  14
left:  0.8574999999999999
right:  1
Prob 0: 0.984422601128019
Prob 1: 0.015577398871981046
F(a_max)**2:  (0.9845340796468466+0j)
Success Prob:  0.984422601128019
------------------
x: 0.959375
d:  14
left:  0.91875
right:  1
Prob 0: 0.7245804600911738
Prob 1: 0.2754195399088241
F(a_max)**2:  (0.7247094310732473+0j)
Success Prob:  0.7245804600911738
------------------
x: 0.9746874999999999
d:  14
left:  0.949375
right:  1
Prob 0: 0.5939095561204485
Prob 1: 0.40609044387955173
F(a_max)**2:  (0.59404105552365+0j)
Success Prob:  0.5939095561204485
Not steep enough!
------------------
x: 0.9746874999999999
d:  18
left:  0.939375
right:  1